# pymongo

## python binding for MongoDB
<br>

# Table of Contets

### 1. **Connection**

- [Run MongoDB Server](#run_server)
- [Connect to MongoDB](#connection)

### 2. Basic Usage

- [Create](#create)
- [Read](#read)
- [Update](#update)
- [Delete](#delete)

### 3. Query

- [Query](#query)

### 4. Indexing

- [Indexing](#index)

### 5. Sorting & Limit

- [Sorting](#sort)
- [Limit](#limit)

### 6. MapReduce

- [Basic MapReduce](#basic_mapreduce)

### 7. Working with Pandas

- [MongoDB cursor to pandas df](#mongodb_cursor_to_df)

### 8. ETC

- [Total Count of collections](#count_collection)

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

---

<a id="run_server"></a>
# Run MongoDB Server

### run mongodb server command
"C:\Program Files\MongoDB\Server\3.4\bin\mongod.exe" --dbpath "C:\Codes\mongodb"

In [1]:
import pymongo
from pymongo import MongoClient

from pprint import pprint

---

<a id="connection"></a>
## Connect to MongoDB

### Create Client

In [2]:
mongo_client = MongoClient('localhost',27017)

In [3]:
mongo_client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

### connect to DB

In [4]:
mongo_db = mongo_client['test_db']

In [5]:
mongo_db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_db')

### connect to collection

In [6]:
mongo_collection = mongo_db['node_sample']

In [7]:
mongo_collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_db'), 'node_sample')

---

<a id="read"></a>
## read collection data

**using find_one**

In [8]:
data = mongo_collection.find_one()

In [9]:
# return 1st document of collection
data

{'_id': ObjectId('5a1297c4aeea91055838099c'),
 'comment': 'new comment',
 'order': 2,
 'title': 'also changed'}

**using find**

In [10]:
data = mongo_collection.find()

In [11]:
data

In [12]:
for datum in data:
    print(type(datum))
    pprint(datum)

<class 'dict'>
{'_id': ObjectId('5a1297c4aeea91055838099c'),
 'comment': 'new comment',
 'order': 2,
 'title': 'also changed'}
<class 'dict'>
{'_id': ObjectId('5a1297c4aeea91055838099d'),
 'comment': 'value3',
 'order': 3,
 'title': 'whoever'}
<class 'dict'>
{'_id': ObjectId('5a1a226baeea9118e02d64cc'),
 'comment': 'insert by insert_many',
 'order': 6,
 'title': 'no matter'}
<class 'dict'>
{'_id': ObjectId('5a1a5a595b005df5c002db3c'),
 'comment': 'insert by insert_many',
 'order': 1,
 'title': 'no matter'}
<class 'dict'>
{'_id': ObjectId('5a1a5a635b005df5c002db3f'),
 'comment': 'insert by insert_many',
 'order': 1,
 'title': 'no matter what ????'}


---

<a id="query"></a>
## Query

In [13]:
query = {
    'order':2,
}

data = mongo_collection.find(query)

In [14]:
data

In [15]:
for datum in data:
    print(type(datum))
    pprint(datum)

<class 'dict'>
{'_id': ObjectId('5a1297c4aeea91055838099c'),
 'comment': 'new comment',
 'order': 2,
 'title': 'also changed'}


query with **multiple condition**

In [16]:
query = {
    '$and':[
        {'order':2},
        {'title':'whenever'},
    ]
}

data = mongo_collection.find_one(query)

In [17]:
pprint(data)

None


query only **typical column**

In [18]:
query = {    
    'order':2,
}

cols = {'order':1, 'comment':1}

data = mongo_collection.find_one(query, cols)

id pk field added by default

In [19]:
pprint(data)

{'_id': ObjectId('5a1297c4aeea91055838099c'),
 'comment': 'new comment',
 'order': 2}


In [20]:
query = {    
    'order':2,
}

# force remove _id field
cols = {
    '_id':0,
    'order':1, 
    'comment':1,
}

data = mongo_collection.find_one(query, cols)

In [21]:
data

{'comment': 'new comment', 'order': 2}

---

<a id="create"></a>
## Create

Insert using **insert_one**

In [22]:
document = {
    'order':4,
    'title':'no matter',
    'comment':'insert by insert_one',
}

mongo_collection.insert_one(document)

Bulk insert using **insert_many**

In [23]:
documents = [
    {
    'order':5,
    'title':'no matter',
    'comment':'insert by insert_many',
    },
    {
    'order':6,
    'title':'no matter',
    'comment':'insert by insert_many',
    },
]

In [24]:
type(documents)

list

In [25]:
mongo_collection.insert_many(documents)

## Modify document

In [26]:
document = mongo_collection.find_one({'order':2})

In [27]:
document

{'_id': ObjectId('5a1297c4aeea91055838099c'),
 'comment': 'new comment',
 'order': 2,
 'title': 'also changed'}

only change **typical parts**

In [28]:
mongo_collection.update(
    {'order':2},
    {'$set':
        {'comment':'new comment', 'title': 'also changed'},        
    },
)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  after removing the cwd from sys.path.


{'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}

**replace document**

replace = {
    'order':1.0,
    'title':'new title',
    'comment':'new comment',
}

mongo_collection.update(
    {'order':1},
    replace,
    upsert=True,   # when upser=True, creates document when query result is None
)

---

<a id="delete"></a>
# Delete

using **delete_one**

In [30]:
query = {
    'order':1.1,
}

mongo_collection.delete_one(query)

In [31]:
data = mongo_collection.find()

for datum in data:
    pprint(datum)

{'_id': ObjectId('5a1297c4aeea91055838099c'),
 'comment': 'new comment',
 'order': 2,
 'title': 'also changed'}
{'_id': ObjectId('5a1297c4aeea91055838099d'),
 'comment': 'value3',
 'order': 3,
 'title': 'whoever'}
{'_id': ObjectId('5a1a226baeea9118e02d64cc'),
 'comment': 'insert by insert_many',
 'order': 6,
 'title': 'no matter'}
{'_id': ObjectId('5a1a5a595b005df5c002db3c'),
 'comment': 'new comment',
 'order': 1.0,
 'title': 'new title'}
{'_id': ObjectId('5a1a5a635b005df5c002db3f'),
 'comment': 'insert by insert_many',
 'order': 1,
 'title': 'no matter what ????'}
{'_id': ObjectId('5a1a9120aeea912a84277acf'),
 'comment': 'insert by insert_one',
 'order': 4,
 'title': 'no matter'}
{'_id': ObjectId('5a1a9120aeea912a84277ad0'),
 'comment': 'insert by insert_many',
 'order': 5,
 'title': 'no matter'}
{'_id': ObjectId('5a1a9120aeea912a84277ad1'),
 'comment': 'insert by insert_many',
 'order': 6,
 'title': 'no matter'}


using **delete_many**

In [32]:
query = {
    '$and':[
        {'order':{'$gte':4}},
        {'order':{'$lte':5}},
    ]
}

mongo_collection.delete_many(query)

In [33]:
data = mongo_collection.find()

for datum in data:
    pprint(datum)

{'_id': ObjectId('5a1297c4aeea91055838099c'),
 'comment': 'new comment',
 'order': 2,
 'title': 'also changed'}
{'_id': ObjectId('5a1297c4aeea91055838099d'),
 'comment': 'value3',
 'order': 3,
 'title': 'whoever'}
{'_id': ObjectId('5a1a226baeea9118e02d64cc'),
 'comment': 'insert by insert_many',
 'order': 6,
 'title': 'no matter'}
{'_id': ObjectId('5a1a5a595b005df5c002db3c'),
 'comment': 'new comment',
 'order': 1.0,
 'title': 'new title'}
{'_id': ObjectId('5a1a5a635b005df5c002db3f'),
 'comment': 'insert by insert_many',
 'order': 1,
 'title': 'no matter what ????'}
{'_id': ObjectId('5a1a9120aeea912a84277ad1'),
 'comment': 'insert by insert_many',
 'order': 6,
 'title': 'no matter'}


---

<a id="count_collection"></a>
## Total Count of collections

In [34]:
mongo_collection.count()

6

<a id="sort"></a>
## Sorting

In [35]:
data = mongo_collection.find().sort('order', pymongo.ASCENDING)

In [36]:
for datum in data:
    pprint(datum)

{'_id': ObjectId('5a1a5a595b005df5c002db3c'),
 'comment': 'new comment',
 'order': 1.0,
 'title': 'new title'}
{'_id': ObjectId('5a1a5a635b005df5c002db3f'),
 'comment': 'insert by insert_many',
 'order': 1,
 'title': 'no matter what ????'}
{'_id': ObjectId('5a1297c4aeea91055838099c'),
 'comment': 'new comment',
 'order': 2,
 'title': 'also changed'}
{'_id': ObjectId('5a1297c4aeea91055838099d'),
 'comment': 'value3',
 'order': 3,
 'title': 'whoever'}
{'_id': ObjectId('5a1a226baeea9118e02d64cc'),
 'comment': 'insert by insert_many',
 'order': 6,
 'title': 'no matter'}
{'_id': ObjectId('5a1a9120aeea912a84277ad1'),
 'comment': 'insert by insert_many',
 'order': 6,
 'title': 'no matter'}


In [37]:
data = mongo_collection.find().sort('order', pymongo.DESCENDING)

In [38]:
for datum in data:
    pprint(datum)

{'_id': ObjectId('5a1a9120aeea912a84277ad1'),
 'comment': 'insert by insert_many',
 'order': 6,
 'title': 'no matter'}
{'_id': ObjectId('5a1a226baeea9118e02d64cc'),
 'comment': 'insert by insert_many',
 'order': 6,
 'title': 'no matter'}
{'_id': ObjectId('5a1297c4aeea91055838099d'),
 'comment': 'value3',
 'order': 3,
 'title': 'whoever'}
{'_id': ObjectId('5a1297c4aeea91055838099c'),
 'comment': 'new comment',
 'order': 2,
 'title': 'also changed'}
{'_id': ObjectId('5a1a5a635b005df5c002db3f'),
 'comment': 'insert by insert_many',
 'order': 1,
 'title': 'no matter what ????'}
{'_id': ObjectId('5a1a5a595b005df5c002db3c'),
 'comment': 'new comment',
 'order': 1.0,
 'title': 'new title'}


<a id="limit"></a>
## Limit

In [39]:
data = mongo_collection.find().sort('order', pymongo.DESCENDING).limit(3)

In [40]:
for datum in data:
    pprint(datum)

{'_id': ObjectId('5a1a9120aeea912a84277ad1'),
 'comment': 'insert by insert_many',
 'order': 6,
 'title': 'no matter'}
{'_id': ObjectId('5a1a226baeea9118e02d64cc'),
 'comment': 'insert by insert_many',
 'order': 6,
 'title': 'no matter'}
{'_id': ObjectId('5a1297c4aeea91055838099d'),
 'comment': 'value3',
 'order': 3,
 'title': 'whoever'}


---

<a id="index"></a>
# Indexing

In [41]:
mongo_collection.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'test_db.node_sample', 'v': 2},
 'order_1_title_1_comment_1': {'key': [('order', 1),
   ('title', 1),
   ('comment', 1)],
  'ns': 'test_db.node_sample',
  'v': 2}}

### set Index of collection

In [42]:
result = mongo_collection.create_index(
    [
        ('order', pymongo.ASCENDING),
        ('title', pymongo.ASCENDING),
        ('comment', pymongo.ASCENDING),
    ]
)

### get index information from collection

In [43]:
pprint(mongo_collection.index_information())

{'_id_': {'key': [('_id', 1)], 'ns': 'test_db.node_sample', 'v': 2},
 'order_1_title_1_comment_1': {'key': [('order', 1),
                                       ('title', 1),
                                       ('comment', 1)],
                               'ns': 'test_db.node_sample',
                               'v': 2}}


---

<a id="basic_mapreduce"></a>
# Basic MapReduce

In [44]:
from bson.code import Code

In [45]:
map = Code("function () {"
           "  this.order.forEach(function(z) {"
           "    emit(z, 1);"
           "  });"
           "}")

In [46]:
map

Code('function () {  this.order.forEach(function(z) {    emit(z, 1);  });}', None)

In [47]:
reduce = Code("function (key, values) {"
              "  var total = 0;"
              "  for (var i = 0; i < values.length; i++) {"
              "    total += values[i];"
              "  }"
              "  return total;"
              "}")

In [48]:
reduce

Code('function (key, values) {  var total = 0;  for (var i = 0; i < values.length; i++) {    total += values[i];  }  return total;}', None)

### mapreduce example on mongo shell

In [49]:
"""
function map() {emit(this.order, this.comment)}
function reduce(key, value) { return value.join() }

db.node_sample.mapReduce(map, reduce, {out: {inline:1}});
"""

'\nfunction map() {emit(this.order, this.comment)}\nfunction reduce(key, value) { return value.join() }\n\ndb.node_sample.mapReduce(map, reduce, {out: {inline:1}});\n'

---

<a id="mongodb_cursor_to_df"></a>
# MongoDB cursor to pandas df

In [57]:
import pandas as pd

In [68]:
mongo_coll = mongo_collection.find()

### mongodb cursor object to list of dictionary

In [69]:
mongo_coll_list = list(mongo_coll)

In [74]:
mongo_coll_list[:3]

[{'_id': ObjectId('5a1297c4aeea91055838099c'),
  'comment': 'new comment',
  'order': 2,
  'title': 'also changed'},
 {'_id': ObjectId('5a1297c4aeea91055838099d'),
  'comment': 'value3',
  'order': 3,
  'title': 'whoever'},
 {'_id': ObjectId('5a1a226baeea9118e02d64cc'),
  'comment': 'insert by insert_many',
  'order': 6,
  'title': 'no matter'}]

In [71]:
df = pd.DataFrame(mongo_coll_list)

In [72]:
df

,_id,comment,order,title
0,5a1297c4aeea91055838099c,new comment,2.0,also changed
1,5a1297c4aeea91055838099d,value3,3.0,whoever
2,5a1a226baeea9118e02d64cc,insert by insert_many,6.0,no matter
3,5a1a5a595b005df5c002db3c,new comment,1.0,new title
4,5a1a5a635b005df5c002db3f,insert by insert_many,1.0,no matter what ????
5,5a1a9120aeea912a84277ad1,insert by insert_many,6.0,no matter
